### Eager executation basics

In [1]:
import tensorflow as tf
tf.enable_eager_execution()

In [5]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)


#### Numpy Compatibility
- TensorFlow operations automatically convert NumPy ndarrays to Tensors.
- NumPy operations automatically convert Tensors to NumPy ndarrays.

Tensors can be explicitly converted to NumPy ndarrays by invoking the .numpy() method on them

In [7]:
import numpy as np
ndarray = np.ones([3, 3])
tensor = tf.multiply(ndarray, 42)
print(tensor)
print(tensor.numpy())

tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


#### GPU acceleration
The **Tensor.device** property provides a fully qualified string name of the device hosting the contents of the tensor.

The string ends with **GPU:N** if the tensor is placed on the N-th GPU on the host.

In [9]:
def time_matmul(x):
    %timeit tf.matmul(x, x)
with tf.device('CPU:0'):
    x = tf.random_uniform([1000, 1000])
    assert x.device.endswith('CPU:0')
    time_matmul(x)
if tf.test.is_gpu_available():
    with tf.device('GPU:0'):
        x = tf.random_uniform([1000, 1000])
        assert x.device.endswith('GPU:0')
        time_matmul(x)

32.1 ms ± 2.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Datasets
This section demonstrates the use of the tf.data.Dataset API to build pipelines to feed data to your model. It covers:
- Creating a Dataset.
- Iteration over a Dataset with eager execution enabled.
##### create a source Dataset
Create a source dataset using one of the factory functions like `Dataset.from_tensors`, `Dataset.from_tensor_slices` or using objects that read from files like `TextLineDataset` or `TFRecordDataset`
##### Apply transformations
Use the transformations functions like `map, batch, shuffle` etc. to apply transformations to the records of the dataset.
##### Iterate
When eager execution is enabled `Dataset` objects support iteration. If you're familiar with the use of Datasets in TensorFlow graphs, note that there is no need for calls to `Dataset.make_one_shot_iterator()` or `get_next()` calls.

In [10]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)
for x in ds_tensors:
  print(x)

tf.Tensor([1 4], shape=(2,), dtype=int32)
tf.Tensor([ 9 25], shape=(2,), dtype=int32)
tf.Tensor([36 16], shape=(2,), dtype=int32)


### Automatic differentiation and gradient tape
TensorFlow provides the tf.GradientTape API for automatic differentiation - computing the gradient of a computation with respect to its input variables. Tensorflow "records" all operations executed inside the context of a tf.GradientTape onto a "tape".

In [11]:
x = tf.ones((2, 2))
with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)
dz_dx = t.gradient(z, x)
for i in [0, 1]:
    for j in [ 0, 1]:
        assert dz_dx[i][j].numpy() == 8.

By default, the resources held by a GradientTape are released as soon as GradientTape.gradient() method is called. To compute multiple gradients over the same computation, create a persistent gradient tape. This allows multiple calls to the gradient() method. as resources are released when the tape object is garbage collected

In [15]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
  t.watch(x)
  y = x * x
  z = y * y
dz_dx = t.gradient(z, x)  # 108.0 (4*x^3 at x = 3)
dy_dx = t.gradient(y, x)  # 6.0
del t  # Drop the reference to the tape

#### Higher-order gradients
Operations inside of the **GradientTape** context manager are recorded for automatic differentiation. If gradients are computed in that context, then the gradient computation is recorded as well.

In [16]:
x = tf.Variable(1.)
with tf.GradientTape() as t:
    with tf.GradientTape() as t2:
        y = x*x*x
        dy_dx = t2.gradient(y, x)
    d2y_dx2 = t.gradient(dy_dx, x)
print(dy_dx.numpy(), d2y_dx2.numpy())

3.0 6.0


### Custom training: Basics
Tensors in TensorFlow are immutable stateless objects

A Variable is an object which stores a value and, when used in a TensorFlow computation, will implicitly read from this stored value. There are operations (`tf.assign_sub, tf.scatter_update`, etc) which manipulate the value stored in a TensorFlow variable
1. Define the model.
2. Define a loss function.
3. Obtain training data.
4. Run through the training data and use an "optimizer" to adjust the variables to fit the data.
#### Define the model

In [17]:
class Model(object):
    def __init__(self):
        self.W = tf.Variable(5.0)
        self.b = tf.Variable(0.0)   
    def __call__(self, x):
        return self.W * x + self.b

#### define a loss function

In [18]:
def loss(predicted_y, desired_y):
    return tf.reduce_mean(tf.square(predicted_y - desired_y))

#### define a training loop

In [19]:
def train(model, inputs, outputs, learning_rate):
    with tf.GradientTape() as t:
        current_loss = loss(model(inputs), outputs)
    dW, db = t.gradient(current_loss, [model.W, model.b])
    model.W.assign_sub(learning_rate * dW)
    model.b.assign_sub(learning_rate * db)

In [20]:
TRUE_W = 3.0
TRUE_b = 2.0
NUM_EXAMPLES = 1000

inputs  = tf.random_normal(shape=[NUM_EXAMPLES])
noise   = tf.random_normal(shape=[NUM_EXAMPLES])
outputs = inputs * TRUE_W + TRUE_b + noise
model = Model()
Ws, bs = [], [ ]
epochs = range(10)
for epoch in epochs:
    Ws.append(model.W.numpy())
    bs.append(model.b.numpy())
    current_loss = loss(model(inputs), outputs)
    train(model, inputs, outputs, learning_rate=0.1)
    print(epoch, Ws[-1], bs[-1], current_loss)

0 5.0 0.0 tf.Tensor(8.994057, shape=(), dtype=float32)
1 4.5929465 0.39435548 tf.Tensor(6.1051965, shape=(), dtype=float32)
2 4.268388 0.7099353 tf.Tensor(4.262118, shape=(), dtype=float32)
3 4.009605 0.9624752 tf.Tensor(3.0862277, shape=(), dtype=float32)
4 3.8032672 1.1645677 tf.Tensor(2.3359957, shape=(), dtype=float32)
5 3.638746 1.3262901 tf.Tensor(1.857332, shape=(), dtype=float32)
6 3.5075667 1.4557066 tf.Tensor(1.5519314, shape=(), dtype=float32)
7 3.402972 1.5592705 tf.Tensor(1.3570745, shape=(), dtype=float32)
8 3.3195744 1.6421462 tf.Tensor(1.2327466, shape=(), dtype=float32)
9 3.2530777 1.7084663 tf.Tensor(1.1534189, shape=(), dtype=float32)


### Custom layers
#### Implementing custom layers
The best way to implement your own layer is extending the tf.keras.Layer class and implementing: ` __init__` , where you can do all input-independent initialization ` build`, where you know the shapes of the input tensors and can do the rest of the initialization `call`, where you do the forward computation

In [1]:
import tensorflow as tf
tf.enable_eager_execution()

In [2]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
    def build(self, input_shape):
        self.kernel = self.add_variable('kernel', 
                                       shape = [int(input_shape[-1]), 
                                               self.num_outputs])
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)
layer = MyDenseLayer(10)
print(layer(tf.ones([3, 5])))
print(layer.variables)

tf.Tensor(
[[ 1.2422413   0.57676595 -0.7736479   1.4771876   1.9684818   0.95305157
   0.78279495  1.1986037   0.8111862  -1.7414144 ]
 [ 1.2422413   0.57676595 -0.7736479   1.4771876   1.9684818   0.95305157
   0.78279495  1.1986037   0.8111862  -1.7414144 ]
 [ 1.2422413   0.57676595 -0.7736479   1.4771876   1.9684818   0.95305157
   0.78279495  1.1986037   0.8111862  -1.7414144 ]], shape=(3, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 0.5884097 , -0.55847716, -0.20097992,  0.05256915,  0.3180158 ,
         0.06278121, -0.1324001 ,  0.58853537,  0.18911612, -0.31677678],
       [ 0.31581897,  0.01617968,  0.31303602,  0.46660286,  0.15058666,
         0.05965292,  0.2888443 ,  0.61008686, -0.16126266, -0.34060073],
       [ 0.07550091,  0.5519212 , -0.43838778,  0.630462  ,  0.39130014,
        -0.24216768,  0.37751848,  0.16110808,  0.3274299 ,  0.00487834],
       [ 0.2899918 ,  0.2021476 , -0.2346243 ,  0.35973012,  0.60

#### Models: composing layers

In [8]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name = '')
        filters1, filters2, filters3 = filters
        
        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()
        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()
        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()
    def call(self, input_tensor, training = False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training = training)
        x = tf.nn.relu(x)
        x = self.conv2b(x)
        x = self.bn2b(x, training = training)
        x = tf.nn.relu(x)
        x = self.conv2c(x)
        x = self.bn2c(x, training = training)
        x += input_tensor
        return tf.nn.relu(x)

block = ResnetIdentityBlock(1, [1, 2, 3])
print(block(tf.zeros([1, 2, 3, 3])))
print([x.name for x in block.variables])

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block_2/conv2d_15/kernel:0', 'resnet_identity_block_2/conv2d_15/bias:0', 'resnet_identity_block_2/batch_normalization_15/gamma:0', 'resnet_identity_block_2/batch_normalization_15/beta:0', 'resnet_identity_block_2/conv2d_16/kernel:0', 'resnet_identity_block_2/conv2d_16/bias:0', 'resnet_identity_block_2/batch_normalization_16/gamma:0', 'resnet_identity_block_2/batch_normalization_16/beta:0', 'resnet_identity_block_2/conv2d_17/kernel:0', 'resnet_identity_block_2/conv2d_17/bias:0', 'resnet_identity_block_2/batch_normalization_17/gamma:0', 'resnet_identity_block_2/batch_normalization_17/beta:0', 'resnet_identity_block_2/batch_normalization_15/moving_mean:0', 'resnet_identity_block_2/batch_normalization_15/moving_variance:0', 'resnet_identity_block_2/batch_normalization_16/moving_mean:0', 'resnet_identity_block_2/batch_normalization_16/movi

### Custom Training: walkthrough
#### Tensorflow programming
This tutorial is structured like many TensorFlow programs:
1. Import and parse the data sets.
2. Select the type of model.
3. Train the model.
4. Evaluate the model's effectiveness.
5. Use the trained model to make predictions.

##### Download the dataset
##### Inspect the data

In [11]:
import os
train_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"
train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

8192/2194 [================================================================================================================] - 0s 0us/step


In [ ]:
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
feature_names = column_names[:-1]
label_name = column_names[-1]
class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']

##### create a `tf.data.Dataset`
TensorFlow's `Dataset` API handles many common cases for loading data into a model. This is a high-level API for reading data and transforming it into a form used for training.

Since the dataset is a CSV-formatted text file, use the `make_csv_dataset` function to parse the data into a suitable format. Since this function generates data for training models, the default behavior is to shuffle the data `(shuffle=True, shuffle_buffer_size=10000)`, and repeat the dataset forever `(num_epochs=None)`.

In [20]:
batch_size = 32
train_dataset = tf.contrib.data.make_csv_dataset(
train_dataset_fp, batch_size, column_names = column_names, 
label_name = label_name, num_epochs = 1)
feature, label = next(iter(train_dataset))

In [28]:
def pack_feature_vector(feature, labels):
    features = tf.stack(list(feature.values()), axis = 1)
    return features, labels
train_dataset = train_dataset.map(pack_feature_vector)

##### Select the type of model

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape = (4, )),
    tf.keras.layers.Dense(10, activation = tf.nn.relu), 
    tf.keras.layers.Dense(3)
])
# training the model
def loss(model, x, y):
    y_ = model(x)
    return tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

##### Create an optimizer
An optimizer applies the computed gradients to the model's variables to minimize the loss function

In [31]:
feature, label = next(iter(train_dataset))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
global_step = tf.Variable(0)
loss_value, grads = grad(model, feature, label)
print("Step: {}, Initial Loss: {}".format(global_step.numpy(),
                                          loss_value.numpy()))
optimizer.apply_gradients(zip(grads, model.variables), global_step)
print("Step: {},         Loss: {}".format(global_step.numpy(),
                                          loss(model, feature, label).numpy()))

Step: 0, Initial Loss: 1.180802822113037
Step: 1,         Loss: 1.1679582595825195


##### Training loop
The following code block sets up these training steps:
1. Iterate each epoch. An epoch is one pass through the dataset.
2. Within an epoch, iterate over each example in the training Dataset grabbing its features (x) and label (y).
3. Using the example's features, make a prediction and compare it with the label. Measure the inaccuracy of the prediction and use that to calculate the model's loss and gradients.
4. Use an optimizer to update the model's variables.
5. Keep track of some stats for visualization.
6. Repeat for each epoch.

In [35]:
train_loss_results = []
train_accuracy_results = [ ]
num_epochs = 20
for epoch in range(num_epochs):
    epoch_loss_avg = tf.metrics.Mean()
    epoch_accuracy = tf.metrics.Accuracy()
    for x, y in train_dataset:
        loss_value, grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.variables), 
                                 global_step)
        epoch_loss_avg(loss_value)
        epoch_accuracy(tf.argmax(model(x), axis = 1, output_type=tf.int32), y)
        train_loss_results.append(epoch_loss_avg.result())
        train_accuracy_results.append(epoch_accuracy.result())

AttributeError: module 'tensorflow._api.v1.metrics' has no attribute 'Mean'

In [ ]:
tf.metrics.